# Telco Customer Churn - Python Analysis
### Main Objective:
1. Exploratory Data Anlaysis
2. Data Pre-processing
3. Model Building
4. Model Evaluation
---

### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import mysql.connector
from mysql.connector import Error

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import classification_report

#### Load the data

In [2]:
# DB Config
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': 'root',
    'database': 'telco_churn'
}

In [3]:
# SQL Query to load the data
sql_query = """
SELECT
    c.customerID,
    c.gender,
    c.SeniorCitizen,
    c.Partner,
    c.Dependents,
    a.tenure,
    a.Contract,
    a.PaperlessBilling,
    a.PaymentMethod,
    a.MonthlyCharges,
    a.TotalCharges,
    s.PhoneService,
    s.MultipleLines,
    s.InternetService,
    s.OnlineSecurity,
    s.OnlineBackup,
    s.DeviceProtection,
    s.TechSupport,
    s.StreamingTV,
    s.StreamingMovies,
    ch.Churn
FROM
    customers c
JOIN
    account_info a ON c.customerID = a.customerID
JOIN
    services s ON c.customerID = s.customerID
JOIN
    churn_data ch ON c.customerID = ch.customerID;
"""

# Connect to the database and load the data
# Create a variable 'df' to hold the data
df = None
conn = None # Initialize connection to None

try:
    # Establish the connection
    conn = mysql.connector.connect(**db_config)
    
    if conn.is_connected():
        print("Successfully connected to the MySQL database.")
        
        # Use pandas to execute the query and load data into a DataFrame
        df = pd.read_sql(sql_query, conn)
        
        print("Data successfully loaded into a pandas DataFrame.")

except Error as e:
    print(f"Error connecting to MySQL or fetching data: {e}")

finally:
    # Make sure to close the connection
    if conn and conn.is_connected():
        conn.close()
        print("MySQL connection is closed.")

Successfully connected to the MySQL database.
Data successfully loaded into a pandas DataFrame.
MySQL connection is closed.


C:\Users\dkpc2\AppData\Local\Temp\ipykernel_8820\3995519853.py:48: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, conn)


In [4]:
# Check if the dataset is loaded properly
if df is not None:
    print("\n--- Data Verification ---")
    
    # Check the number of rows and columns loaded
    # This should show approximately (7043, 21)
    print(f"The DataFrame has {df.shape[0]} rows and {df.shape[1]} columns.")
    
    # Display the first 5 rows to see what the data looks like
    print("\nFirst 5 rows of the dataset:")
    display(df.head())
    
    # Display summary info to check for null values and data types
    print("\nDataFrame Info:")
    df.info()


--- Data Verification ---
The DataFrame has 7043 rows and 21 columns.

First 5 rows of the dataset:


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,...,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Churn
0,0002-ORFBO,Female,0,Yes,Yes,9,One year,Yes,Mailed check,65.6,...,Yes,No,DSL,No,Yes,No,Yes,Yes,No,No\r
1,0003-MKNFE,Male,0,No,No,9,Month-to-month,No,Mailed check,59.9,...,Yes,Yes,DSL,No,No,No,No,No,Yes,No\r
2,0004-TLHLJ,Male,0,No,No,4,Month-to-month,Yes,Electronic check,73.9,...,Yes,No,Fiber optic,No,No,Yes,No,No,No,Yes
3,0011-IGKFF,Male,1,Yes,No,13,Month-to-month,Yes,Electronic check,98.0,...,Yes,No,Fiber optic,No,Yes,Yes,No,Yes,Yes,Yes
4,0013-EXCHZ,Female,1,Yes,No,3,Month-to-month,Yes,Mailed check,83.9,...,Yes,No,Fiber optic,No,No,No,Yes,Yes,No,Yes



DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   Contract          7043 non-null   object 
 7   PaperlessBilling  7043 non-null   object 
 8   PaymentMethod     7043 non-null   object 
 9   MonthlyCharges    7043 non-null   float64
 10  TotalCharges      7032 non-null   float64
 11  PhoneService      7043 non-null   object 
 12  MultipleLines     7043 non-null   object 
 13  InternetService   7043 non-null   object 
 14  OnlineSecurity    7043 non-null   object 
 15  OnlineBackup      7043 non-null   object 
 16  DeviceProtection  7043 no

#### Initial Profiling

In [5]:
missing_val = df.isna().sum()
missing_val

customerID           0
gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
Contract             0
PaperlessBilling     0
PaymentMethod        0
MonthlyCharges       0
TotalCharges        11
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Churn                0
dtype: int64

There's 11 missing values on TotalCharges column. We will drop the missing values.

In [6]:
df = df.dropna(axis=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7032 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7032 non-null   object 
 1   gender            7032 non-null   object 
 2   SeniorCitizen     7032 non-null   int64  
 3   Partner           7032 non-null   object 
 4   Dependents        7032 non-null   object 
 5   tenure            7032 non-null   int64  
 6   Contract          7032 non-null   object 
 7   PaperlessBilling  7032 non-null   object 
 8   PaymentMethod     7032 non-null   object 
 9   MonthlyCharges    7032 non-null   float64
 10  TotalCharges      7032 non-null   float64
 11  PhoneService      7032 non-null   object 
 12  MultipleLines     7032 non-null   object 
 13  InternetService   7032 non-null   object 
 14  OnlineSecurity    7032 non-null   object 
 15  OnlineBackup      7032 non-null   object 
 16  DeviceProtection  7032 non-null   object 
 17  

In [7]:
# Check duplicate
df.duplicated().sum()

np.int64(0)

No duplicate found on the dataset.

#### Exploratory Data Analysis

In [13]:
# Gender distribution
cust_gender = df.groupby('gender')[['gender']].count()
cust_gender = cust_gender.rename(columns={
    'gender' : 'Count of gender'
}).reset_index()
cust_gender

,gender,Count of gender
0,Female,3483
1,Male,3549


In [14]:
# Distribution of internet services
internet_services = df.groupby('InternetService')[['InternetService']].count()
internet_services = internet_services.rename(columns={
    'InternetService' : 'Count of Internet Service'
}).reset_index()
internet_services

,InternetService,Count of Internet Service
0,DSL,2416
1,Fiber optic,3096
2,No,1520


In [16]:
# Customers per contract type
cust_contract = df.groupby('Contract')[['Contract']].count()
cust_contract = cust_contract.rename(columns={
    'Contract' : 'Count of Contract'
}).reset_index()
cust_contract

,Contract,Count of Contract
0,Month-to-month,3875
1,One year,1472
2,Two year,1685


In [ ]:
# Analyze the churn rate by Contract type.
# SELECT
#     a.Contract,
#     COUNT(ch.customerID) AS total_customers,
#     SUM(CASE WHEN ch.Churn = 'Yes' THEN 1 ELSE 0 END) AS churned_customers,
#     -- Calculate the churn rate as a percentage
#     (SUM(CASE WHEN ch.Churn = 'Yes' THEN 1 ELSE 0 END) * 100.0 / COUNT(ch.customerID)) AS churn_rate_percent
# FROM
#     account_info a
# JOIN
#     churn_data ch ON a.customerID = ch.customerID
# GROUP BY
#     a.Contract
# ORDER BY
#     churn_rate_percent DESC

